In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_milvus import Milvus
from dotenv import load_dotenv
from uuid import uuid4
from pymilvus import (
    MilvusClient,
    DataType,
)

In [26]:
load_dotenv()

True

In [19]:
client = MilvusClient("milvus_demo.db")

In [20]:
# Get HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/nfs/home/marquez/personal/udea/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
URI = "./milvus_example.db" # For Milvus Lite

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

In [22]:
# Add items to vector store

document_1 = Document(
    page_content="La Universidad de Antioquia como institución de servicio público, en cumplimiento de su función social, será siempre un centro de cultura y de ciencia que imparta a los estudiantes formación integral y los capacite para el ejercicio profesional en las diferentes áreas del quehacer humano.",
    metadata={"articulo": "1", "titulo": "primero"},
)

document_2 = Document(
    page_content="La función esencial de la Universidad es la docente-investigativa en los campos de la ciencia, la técnica y la cultura y, ante ésta, las demás son básicamente instrumentales. Por lo tanto, y en cumplimiento de su misión, debe la Universidad:",
    metadata={"articulo": "2", "titulo": "primero"},
)

document_3 = Document(
    page_content="Por lo tanto, y en cumplimiento de su misión, debe la Universidad: Desarrollar en sus estudiantes una actitud científica y crítica que les permita tener acceso a los conocimientos en forma libre y consciente.",
    metadata={"articulo": "2", "titulo": "primero", "literal": "a"},
)

document_4 = Document(
    page_content="Por lo tanto, y en cumplimiento de su misión, debe la Universidad: Contribuir al desarrollo en el estudiante de habilidades que le permitan acceder al proceso de aprendizaje. Para ello procurará descubrirle todas las fuentes posibles de información, como también adiestrarlo en los métodos necesarios para utilizarlas apropiadamente.",
    metadata={"articulo": "2", "titulo": "primero", "literal": "b"},
)

document_5 = Document(
    page_content="Por lo tanto, y en cumplimiento de su misión, debe la Universidad: Proporcionar los elementos necesarios para entender la personal ubicación dentro de la sociedad, los valores culturales de la misma y las responsabilidades ante ella.",
    metadata={"articulo": "2", "titulo": "primero", "literal": "c"},
)

document_6 = Document(
    page_content="La función docente-investigativa estará encaminada, fundamentalmente, a promover la creatividad en los campos de la ciencia, las artes y las técnicas, al estudio crítico y responsable, para buscar solución a los problemas nacionales.",
    metadata={"articulo": "3", "titulo": "primero"},
)

document_7 = Document(
    page_content="El proceso de enseñanza aprendizaje, para el cumplimiento de la misión de la Universidad, es la interrelación de profesores y estudiantes, con la utilización de los medios instrumentales necesarios para que, mediante el aprovechamiento de aptitudes y actitudes, se produzca en el educando el necesario cambio de conducta que todo aprendizaje significa.",
    metadata={"articulo": "4", "titulo": "primero"},
)

document_8 = Document(
    page_content="Para cumplir su objetivo el proceso de formación debe desarrollarse dentro de claros criterios éticos y académicos, de tal forma que se dé un clima favorable donde imperen la razón, el mutuo respeto, la libertad de cátedra y la libertad de aprendizaje. Debe además cultivarse, con respeto por la función humana y social de la educación superior, una actitud de sana crítica, que estimule la búsqueda permanente de nuevas expresiones de la ciencia, la cultura, el arte y nuevas formas de desarrollo social.",
    metadata={"articulo": "5", "titulo": "primero"},
)

document_9 = Document(
    page_content="Se entiende por libertad de cátedra la discrecionalidad que tiene el docente para exponer, según su leal saber y entender y ceñido a los métodos científicos, los conocimientos de su especialidad y la que se reconoce al alumno para controvertir dichas exposiciones dentro de los presupuestos académicos.",
    metadata={"articulo": "6", "titulo": "primero"},
)

document_10 = Document(
    page_content="Se entiende por libertad de aprendizaje la que tiene el estudiante para acceder a todas las fuentes de información científica y para utilizar esa información en el incremento y profundización de sus conocimientos.",
    metadata={"articulo": "7", "titulo": "primero"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f1eb1b39-2176-4b9f-a2c6-5073fac34d2b',
 '621cd1ce-f427-48e3-9fc7-257fe0e01659',
 'c6a3da73-d157-447d-a9ab-be2183797fd9',
 'fae8e137-9e21-453b-ab29-6b186cf14712',
 '17adad59-8b9e-41e2-bd28-106528f7bd31',
 '0f324d47-7bcf-41c2-9659-1eff06064a9c',
 '2d517a9c-3839-48cf-a1cb-d05def178a24',
 '61b9f71a-1524-440b-8783-93160bb7892c',
 '07c7ca35-db69-4099-a484-d779371f8726',
 'd293356d-2286-4008-bfbb-442eeb213e20']

In [23]:
# Query by turning into retriever
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("Cuál es la función esencial de la universidad")

[Document(metadata={'articulo': '2', 'pk': 'b301dded-d36f-4e56-92b0-c8088f553c99', 'titulo': 'primero'}, page_content='La función esencial de la Universidad es la docente-investigativa en los campos de la ciencia, la técnica y la cultura y, ante ésta, las demás son básicamente instrumentales. Por lo tanto, y en cumplimiento de su misión, debe la Universidad:')]

In [31]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

In [36]:
from models.llama3_8b_cpp import Llama3_8B

llm = Llama3_8B(quality="q2", verbose=False)

In [43]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
llm = Llama(
    model_path=hf_hub_download("bartowski/Meta-Llama-3-8B-Instruct-GGUF", "Meta-Llama-3-8B-Instruct-Q2_K.gguf"),  
    n_threads=16,
    n_ctx=2048,
)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from /nfs/home/marquez/.cache/huggingface/hub/models--bartowski--Meta-Llama-3-8B-Instruct-GGUF/snapshots/4ebc4aa83d60a5d6f9e1e1e9272a4d6306d770c1/Meta-Llama-3-8B-Instruct-Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

In [44]:
prompt_template = """
<|system|>
Eres un asistente virtual que te debe dar información y responder preguntas sobre la Universidad de Antioquia. Usa los siguientes documentos como contexto para construir tus respuestas:

{context}

Cita siempre los títulos y artículos del reglamento que utilizas para construir tu respuesta.

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [45]:
llm

In [46]:
rag_chain.invoke("Cuál es la función esencial de la universidad")

TypeError: object of type 'StringPromptValue' has no len()